## Monarch + TorchTitan on SLURM
This example notebook demonstrates how you can easily run and iterate on a distributed training job with Monarch and TorchTitan.

#### Prerequisites
Please make sure your environment is setup for this notebook:
1. Install Monarch nightly: https://github.com/meta-pytorch/monarch/blob/main/scripts/install_nightly.py
2. Install Titan nightly: https://github.com/pytorch/torchtitan?tab=readme-ov-file#nightly-builds
3. Ensure you have a valid Titan model config in the script directory (i.e: https://github.com/pytorch/torchtitan/blob/main/torchtitan/models/llama3/train_configs/debug_model.toml)

### 1. Create your SLURM job
Configure parameters for your cluster:
- num_nodes: Number of nodes to allocate (default: 2)
- gpus_per_node: Number of GPUs per node (default: 8)
- mesh_name: Name for the mesh (default: "mesh0")
- time_limit: Maximum job duration (default: "06:00:00")

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

import logging
from monarch.job import SlurmJob

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(name)s %(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)
logger: logging.Logger = logging.getLogger(__name__)

# Configure job parameters
num_nodes = 2  # assign for your system
gpus_per_node = 8  # adjust for your hardware
mesh_name = "mesh0"

# Create a SLURM job with N nodes
slurm_job = SlurmJob(
    meshes={mesh_name: num_nodes},
    job_name="monarch_example",
    gpus_per_node=gpus_per_node,
    time_limit="06:00:00",
)

### 2. Define your Titan and cluster parameters

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

from torchtitan.train import Trainer
from torchtitan.config import ConfigManager, JobConfig
from monarch.actor import Actor, current_rank, endpoint
from torchtitan.tools.logging import init_logger, logger
import torch
from dataclasses import dataclass
import os
from monarch.utils import setup_env_for_distributed


@dataclass
class RunParams:
    """
        Parameters for your cluster and training job, adjust as needed
    """
    training_steps: int = 50
    model_config = "debug_model.toml"
    dataset = "c4"
    num_nodes = num_nodes
    gpus_per_node = gpus_per_node


class TrainerActor(Actor):
    """
        A simple wrapper class with executes a TorchTitan trainer in a Monarch actor
    """
    def __init__(self, job_config: JobConfig) -> None:
        self.job_config = job_config
        rank = current_rank().rank
        self.uid = f"[trainer_{rank}]"

    @endpoint
    async def start_training(self) -> None:
        init_logger()
        trainer: Trainer | None = None

        try:
            trainer = Trainer(self.job_config)
            logger.info(f"{self.uid} initialized successfully and starting training")
            trainer.train()
        except Exception:
            if trainer:
                trainer.close()
            raise
        else:
            trainer.close()
        finally:
            torch.distributed.destroy_process_group()
            logger.info(f"{self.uid} trainer cleaned up")

def make_job_config() -> JobConfig:
    """
        Create a job config which is digested by TorchTitan, sourced from RunParams
    """
    data_parallel_shard_degree = RunParams.num_nodes * RunParams.gpus_per_node
    output_path = "./outputs"

    script_dir = globals()['_dh'][0]
    default_args = [
        "--job.config_file",
        os.path.join(script_dir, RunParams.model_config),
        "--model.tokenizer_path",
        os.path.join(script_dir, "tokenizer"),
        "--comm.trace_buf_size",
        "0",
        "--metrics.log_freq",
        "1",
        "--parallelism.data_parallel_shard_degree",
        str(data_parallel_shard_degree),
        "--activation_checkpoint.mode",
        "full",
        "--comm.train_timeout_seconds",
        "60",
        "--training.steps",
        str(RunParams.training_steps),
        "--training.dataset",
        RunParams.dataset,
        "--job.dump_folder",
        output_path,
        "--metrics.enable_tensorboard",
    ]

    config_manager = ConfigManager()
    job_config = config_manager.parse_args(default_args)

    return job_config

### 3. Execute your training job
You can make adjustments and run this on the existing SLURM allocations as many times as you would like!

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

async def main():
    job_config = make_job_config()

    try:
        # 1. Get job state and create process mesh
        job_state = slurm_job.state()
        proc_mesh = job_state.mesh0.spawn_procs({"gpus": RunParams.gpus_per_node})
        
        # 2. Configure remote logging behavior
        await proc_mesh.logging_option(
            stream_to_client=True,
            # aggregate_window_sec=None  # Uncomment to disable log batching
        )
        
        # 3. Setup environment for torch.distributed
        await setup_env_for_distributed(proc_mesh)
        
        # 4. Spawn TrainerActor on each GPU
        trainer = proc_mesh.spawn("trainer_actor", TrainerActor, job_config)
        
        # 5. Execute the training job
        await trainer.start_training.call()
        
        logger.info("Training completed successfully!")
        
    except Exception as e:
        logger.error(f"Training workflow failed: {e}")


if __name__ == "__main__":
    await main()

### 4. Cleanup the SLURM job
Once you're done experimenting, free up the allocation

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

# Cancel the SLURM job, releasing all reserved nodes back to the cluster
slurm_job.kill()
logger.info("Job terminated successfully")